In [1]:
%pip install transformers datasets
%pip install accelerate
%pip --version accelerate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
pip 24.1.2 from c:\Users\mehme\anaconda3\envs\ML-DL\Lib\site-packages\pip (python 3.11)

Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset, Dataset

# JSON dosyanızın yolunu belirtin
dataset = load_dataset('ruslanmv/ai-medical-chatbot', 'default', split="train[:100000]")
dataset[0]

{'Description': 'Q. What does abutment of the nerve root mean?',
 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?',
 'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'}

In [4]:
from transformers import AutoTokenizer

# Tokenizer'ı yükleyin
tokenizer = AutoTokenizer.from_pretrained('t5-small')

def tokenize_function(examples):
    # Girdiyi tokenize edin
    encodings = tokenizer(examples['Patient'], padding='max_length', truncation=True, max_length=128)

    # Yanıtı tokenize edin ve labels olarak ekleyin
    encodings['labels'] = tokenizer(examples['Doctor'], padding='max_length', truncation=True, max_length=128)['input_ids']

    return encodings

In [5]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 100000/100000 [00:14<00:00, 6959.92 examples/s]


In [6]:
from datasets import DatasetDict

# Veri setini train ve test setlerine ayırma
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

# DatasetDict oluşturma
dataset_dict = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

In [7]:
from transformers import Trainer, TrainingArguments, AutoModelForSeq2SeqLM

# Modeli yükleyin
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

# Eğitim argümanlarını tanımlayın
training_args = TrainingArguments(
    output_dir = "/results/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
)

# Trainer'ı oluşturun
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test'],
)

# Modeli eğitin
trainer.train()

c:\Users\mehme\anaconda3\envs\ML-DL\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▎         | 500/20000 [00:49<37:31,  8.66it/s]

{'loss': 4.4388, 'grad_norm': 1.7864558696746826, 'learning_rate': 1.95e-05, 'epoch': 0.03}


  5%|▌         | 1000/20000 [01:41<38:47,  8.16it/s] 

{'loss': 3.3327, 'grad_norm': 1.412244439125061, 'learning_rate': 1.9e-05, 'epoch': 0.05}


  8%|▊         | 1500/20000 [02:42<33:49,  9.12it/s]  

...

100%|██████████| 20000/20000 [40:56<00:00,  8.59it/s]

{'loss': 2.68, 'grad_norm': 3.0691440105438232, 'learning_rate': 0.0, 'epoch': 1.0}


                                                     
100%|██████████| 20000/20000 [43:20<00:00,  7.69it/s]

{'eval_loss': 2.4760866165161133, 'eval_runtime': 142.2416, 'eval_samples_per_second': 140.606, 'eval_steps_per_second': 35.151, 'epoch': 1.0}
{'train_runtime': 2600.3355, 'train_samples_per_second': 30.765, 'train_steps_per_second': 7.691, 'train_loss': 2.8542715698242187, 'epoch': 1.0}


TrainOutput(global_step=20000, training_loss=2.8542715698242187, metrics={'train_runtime': 2600.3355, 'train_samples_per_second': 30.765, 'train_steps_per_second': 7.691, 'total_flos': 2706836029440000.0, 'train_loss': 2.8542715698242187, 'epoch': 1.0})

In [8]:
model.to("cpu")
# Örnek giriş
input_text = "I have a headache and I do not feel very good. Please explain doctor."

# Tokenize edin
inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True, max_length=128)

In [9]:
import torch
# Modeli değerlendirme moduna alın
model.eval()

# Yanıt üretin
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=512,       # Maksimum yanıt uzunluğu
        num_beams=2,          # Beam search
        early_stopping=True,  # Erken durdurma
        no_repeat_ngram_size=1 # Tekrarları önlemek için
    )

# Yanıtı decode edin
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated response: {generated_text}")

Generated response: Hello,Thanks for using healthcare magic.I have gone through your query and I can understand what you are saying about the symptoms of a headache which is not very good or that it may be due to an allergic reaction from another person in this case as well but there will also need some help with these problems like anxiety disorder (emotional pain), depression etc...


In [10]:
model.save_pretrained('Chatbot/Python/Models/Models_local/local_medical_assistant_model')
tokenizer.save_pretrained('Chatbot/Python/Models/Models_local/local_medical_assistant_model')

('Chatbot/Python/Models/Models_local/local_medical_assistant_model\\tokenizer_config.json',
 'Chatbot/Python/Models/Models_local/local_medical_assistant_model\\special_tokens_map.json',
 'Chatbot/Python/Models/Models_local/local_medical_assistant_model\\tokenizer.json')